# ZnTrack Parameters with dataclasses

To structure the parameters used in a Node it can be useful to pass them as a dataclass. The following Notebook will illustrate a small Example.
We will define the parameter dataclass and our Node inside a `compute_random_number.py` file and import it into this notebook.

In [1]:
# We work in a temporary directory to avoid cluttering the current directory
import os
import tempfile

temp_dir = tempfile.TemporaryDirectory()
os.chdir(temp_dir.name)

In [2]:
%%writefile compute_random_number.py
import random
import dataclasses
import zntrack

@dataclasses.dataclass
class Parameter:
    start: int
    stop: int
    step: int = 1

class ComputeRandomNumber(zntrack.Node):
    param: Parameter = zntrack.deps()
    number: int = zntrack.outs()

    def run(self):
        self.number = random.randrange(self.param.start, self.param.stop, self.param.step)

Writing compute_random_number.py


In [3]:
import pathlib

from compute_random_number import ComputeRandomNumber, Parameter

import zntrack

In [4]:
!git init

!dvc init -q

Initialized empty Git repository in /private/var/folders/81/0x90c4yd0cz6tz104v9syf580000gn/T/tmp999qe3t1/.git/


In [5]:
project = zntrack.Project()

parameter = Parameter(start=100, stop=200)
with project:
    ComputeRandomNumber(param=parameter)

project.repro()

2024-10-17 08:50:37,573 - INFO: Saving params.yaml


100%|██████████| 1/1 [00:00<00:00, 727.29it/s]


Running stage 'ComputeRandomNumber':
> zntrack run compute_random_number.ComputeRandomNumber --name ComputeRandomNumber
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock nodes/ComputeRandomNumber/.gitignore

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [6]:
print(ComputeRandomNumber.from_rev().number)
print(ComputeRandomNumber.from_rev().param)

189
Parameter(start=100, stop=200, step=1)


The arguments of the dataclass are saved in the `params.yaml` file and can also be modified there.

In [7]:
print(pathlib.Path("params.yaml").read_text())

ComputeRandomNumber:
  param:
    _cls: compute_random_number.Parameter
    start: 100
    step: 1
    stop: 200



In [8]:
temp_dir.cleanup()